In [1]:
import tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

2025-08-01 22:30:12.413122: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-01 22:30:12.414742: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-01 22:30:12.438510: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-01 22:30:12.438576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-01 22:30:12.439429: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# Reading Dataset
df=pd.read_excel('Copy of diet_facts_ollama.xlsx')
df.head()

,message,label
0,"The Mediterranean diet, characterized by high ...",1
1,Low-fat dairy products have been found to incr...,1
2,Adequate consumption of fiber from plant-based...,1
3,The DASH diet (Dietary Approaches to Stop Hype...,1
4,"A diet high in processed meats, such as hot do...",1


In [3]:
# taking Df info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  2000 non-null   object
 1   label    2000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [4]:
df.duplicated().sum()

0

In [5]:
df.isnull().sum()

message    0
label      0
dtype: int64

In [6]:
df['label'].value_counts() # our dataset is not skewed its balanced datset

label
1    1000
0    1000
Name: count, dtype: int64

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/varun-
[nltk_data]     kasnia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
from nltk.corpus import stopwords
stopword=set(stopwords.words('english'))
stopword

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [ ]:
import re
from nltk.stem import WordNetLemmatizer
from gensim.utils import simple_preprocess
import pickle

lematizer=WordNetLemmatizer()

def preprocessor(df):
    words=[]
    for i in df['message']:
        text=simple_preprocess(str(i))
        filtered=[lematizer.lemmatize(word) for word in text if word not in stopword]
        words.append(filtered)

    return words

with open('Lemetizer.pkl','wb') as f:
    pickle.dump(lematizer,f)

In [10]:
sentences=preprocessor(df)
sentences

[['mediterranean',
  'diet',
  'characterized',
  'high',
  'intake',
  'fruit',
  'vegetable',
  'whole',
  'grain',
  'healthy',
  'fat',
  'linked',
  'reduced',
  'risk',
  'cognitive',
  'decline',
  'dementia'],
 ['low',
  'fat',
  'dairy',
  'product',
  'found',
  'increase',
  'risk',
  'type',
  'diabetes',
  'full',
  'fat',
  'dairy',
  'product',
  'pose',
  'risk'],
 ['adequate',
  'consumption',
  'fiber',
  'plant',
  'based',
  'food',
  'help',
  'regulate',
  'blood',
  'sugar',
  'level',
  'improve',
  'insulin',
  'sensitivity',
  'individual',
  'type',
  'diabetes'],
 ['dash',
  'diet',
  'dietary',
  'approach',
  'stop',
  'hypertension',
  'emphasizes',
  'whole',
  'grain',
  'fruit',
  'vegetable',
  'lean',
  'protein',
  'low',
  'fat',
  'dairy',
  'shown',
  'lower',
  'blood',
  'pressure',
  'reduce',
  'risk',
  'cardiovascular',
  'disease'],
 ['diet',
  'high',
  'processed',
  'meat',
  'hot',
  'dog',
  'sausage',
  'linked',
  'increased',
  'ri

In [11]:
from gensim.models import Word2Vec

wv_model=Word2Vec(sentences,vector_size=100,window=10)
wv_model.wv.similar_by_word('diet')

wv_model.save('Word2Vec.model')

In [12]:
wv_model.wv.index_to_key

['diet',
 'food',
 'would',
 'risk',
 'ancient',
 'believed',
 'disease',
 'help',
 'fruit',
 'health',
 'could',
 'said',
 'high',
 'like',
 'mystical',
 'heart',
 'whole',
 'reduce',
 'grain',
 'sacred',
 'vegetable',
 'certain',
 'eating',
 'rich',
 'whispered',
 'one',
 'awaken',
 'sugar',
 'power',
 'dietary',
 'type',
 'god',
 'blood',
 'fatty',
 'fat',
 'linked',
 'cancer',
 'shown',
 'secret',
 'held',
 'essence',
 'diabetes',
 'consumption',
 'fiber',
 'spirit',
 'healthy',
 'low',
 'may',
 'grant',
 'mediterranean',
 'gut',
 'based',
 'consuming',
 'acid',
 'support',
 'processed',
 'level',
 'chronic',
 'omega',
 'within',
 'meal',
 'reduced',
 'plant',
 'protein',
 'improve',
 'people',
 'cardiovascular',
 'eater',
 'lower',
 'consumed',
 'fermented',
 'wisdom',
 'consumer',
 'allowing',
 'summon',
 'weight',
 'realm',
 'fish',
 'whisper',
 'moon',
 'effective',
 'regulate',
 'emphasizes',
 'antioxidant',
 'due',
 'individual',
 'according',
 'hidden',
 'overall',
 'include

In [13]:
wv_model.corpus_count

2000

In [14]:
wv_model.epochs

5

In [ ]:
def avg_word2vec(sentences,wv_model):
    vectors=[]
    for token in sentences:
        
        valid=[word for word in token if word in wv_model.wv.index_to_key]

        if not valid:
            vectors.append(np.zeros(wv_model.vector_size))
        
        else:
            vectors.append(np.mean([wv_model.wv[word] for word in valid],axis=0)) 

    return np.array(vectors)



In [16]:
X=avg_word2vec(sentences,wv_model)
X

array([[ 0.17308405,  0.15018368,  0.00193033, ..., -0.21142897,
         0.02927785, -0.13020591],
       [ 0.14470696,  0.17162852,  0.00033597, ..., -0.24398182,
         0.05514019, -0.14693223],
       [ 0.13967755,  0.13253693, -0.01676287, ..., -0.2472162 ,
         0.04539887, -0.13286169],
       ...,
       [ 0.06602235,  0.0774252 , -0.02518663, ..., -0.25779405,
         0.08193737, -0.11932947],
       [ 0.09181284,  0.10716143, -0.02665969, ..., -0.30450025,
         0.08887307, -0.14326923],
       [ 0.06383471,  0.07081921, -0.02144475, ..., -0.22919308,
         0.07378799, -0.10705572]], dtype=float32)

In [17]:
X=pd.DataFrame(X)

In [18]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.173084,0.150184,0.001930,0.093667,0.210597,-0.488247,0.228762,0.978940,-0.279367,-0.443639,...,0.292320,0.139942,0.264339,0.220115,0.672047,0.313619,0.328596,-0.211429,0.029278,-0.130206
1,0.144707,0.171629,0.000336,0.091038,0.206718,-0.511962,0.236539,0.997418,-0.294162,-0.445910,...,0.304883,0.160158,0.253956,0.217298,0.696451,0.317544,0.340505,-0.243982,0.055140,-0.146932
2,0.139678,0.132537,-0.016763,0.092781,0.210757,-0.506648,0.218314,0.956115,-0.283112,-0.423407,...,0.303601,0.164632,0.235907,0.225871,0.691600,0.301904,0.340193,-0.247216,0.045399,-0.132862
3,0.157643,0.140452,-0.001741,0.089843,0.218519,-0.489386,0.223748,0.977355,-0.282979,-0.445638,...,0.287815,0.145681,0.251870,0.222183,0.677883,0.307126,0.330702,-0.229250,0.035915,-0.132320
4,0.151347,0.188687,0.015648,0.083114,0.181897,-0.457600,0.237334,0.927775,-0.254279,-0.431484,...,0.272940,0.137981,0.259040,0.181274,0.632652,0.286749,0.327519,-0.209779,0.042301,-0.137216


In [19]:
y=df['label']
y.shape

(2000,)

In [20]:
X.shape

(2000, 100)

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
rf_model=RandomForestClassifier()
rf_model.fit(X_train,y_train)
y_pred=rf_model.predict(X_test)
acc=accuracy_score(y_test,y_pred)
classify_report=classification_report(y_test,y_pred)
print(f'Accuracy ->> {acc}\n\n')
print(classify_report)

Accuracy ->> 0.9875


              precision    recall  f1-score   support

           0       1.00      0.98      0.99       201
           1       0.98      1.00      0.99       199

    accuracy                           0.99       400
   macro avg       0.99      0.99      0.99       400
weighted avg       0.99      0.99      0.99       400



In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,classification_report
dt_model=DecisionTreeClassifier()
dt_model.fit(X_train,y_train)
y_pred=dt_model.predict(X_test)
acc=accuracy_score(y_test,y_pred)
classify_report=classification_report(y_test,y_pred)
print(f'Accuracy ->> {acc}\n\n')
print(classify_report)

Accuracy ->> 0.99


              precision    recall  f1-score   support

           0       1.00      0.98      0.99       201
           1       0.98      1.00      0.99       199

    accuracy                           0.99       400
   macro avg       0.99      0.99      0.99       400
weighted avg       0.99      0.99      0.99       400



In [24]:
(X_train.shape[1],)

(100,)

In [25]:
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

In [26]:
dl_model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [27]:
dl_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                6464      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 8577 (33.50 KB)
Trainable params: 8577 (33.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
optmi=tensorflow.optimizers.Adam(learning_rate=0.01)

In [29]:
dl_model.compile(optimizer=optmi,loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
import datetime
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
early_stop=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [31]:
history=dl_model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[early_stop,tensorflow_callback])

Epoch 1/100
50/50 [==============================] - 1s 6ms/step - loss: 0.2790 - accuracy: 0.8963 - val_loss: 0.0325 - val_accuracy: 0.9925
Epoch 2/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0186 - accuracy: 0.9956 - val_loss: 0.0555 - val_accuracy: 0.9850
Epoch 3/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0256 - accuracy: 0.9912 - val_loss: 0.0670 - val_accuracy: 0.9800
Epoch 4/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0275 - accuracy: 0.9900 - val_loss: 0.0766 - val_accuracy: 0.9800
Epoch 5/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0105 - accuracy: 0.9962 - val_loss: 0.0218 - val_accuracy: 0.9925
Epoch 6/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0241 - accuracy: 0.9900 - val_loss: 0.0173 - val_accuracy: 0.9950
Epoch 7/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0153 - accuracy: 0.9937 - val_loss: 0.0180 - val_accuracy: 0.9950
Epoch 8/100
5

In [32]:
dl_model.save('ANN_Model.h5')

/home/varun-kasnia/anaconda3/envs/dl/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
#sno#title#component#soecification#quantity#totalcost#application

In [34]:
# embedding=np.load('word2vec-google-news-300.model.vectors.npy')

In [35]:
# embedding.shape

In [36]:
# from gensim.models import KeyedVectors

# wv=KeyedVectors(vector_size=embedding.shape[1])

# wv.add_vectors(vectors,embedding)

# wv.most_similar('king')